# Практическая работа №2

## 1

In [1]:
!git clone https://github.com/ewatson2/EEL6812_DeepFool_Project

Cloning into 'EEL6812_DeepFool_Project'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 96 (delta 2), reused 1 (delta 1), pack-reused 93 (from 1)
Receiving objects: 100% (96/96), 33.99 MiB | 20.82 MiB/s, done.
Resolving deltas: 100% (27/27), done.


## 2

In [4]:
cd EEL6812_DeepFool_Project/

/content/EEL6812_DeepFool_Project


## 3

In [11]:
import numpy as np
import json
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models
from torchvision.transforms import transforms

## 4

In [12]:
from models.project_models import FC_500_150, LeNet_CIFAR, LeNet_MNIST, Net
from utils.project_utils import get_clip_bounds, evaluate_attack, display_attack

## 5

In [13]:
rand_seed = 17

## 6

In [14]:
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

## 7

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


## 8

In [16]:
mnist_mean = 0.5  # среднее значение для нормализации (/255)
mnist_std = 0.5   # стандартное отклонение для нормализации
mnist_dim = 28    # размерность изображения MNIST (28x28)

# get_clip_bounds вычисляет минимальные и максимальные допустимые значения пикселей после нормализации
mnist_min, mnist_max = get_clip_bounds(mnist_mean, mnist_std, mnist_dim)
mnist_min = mnist_min.to(device)
mnist_max = mnist_max.to(device)

In [17]:
mnist_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize( mean=mnist_mean, std=mnist_std)])

mnist_tf_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mnist_mean, std=mnist_std)])

mnist_tf_inv = transforms.Compose([
    transforms.Normalize(mean=[0.0], std=np.divide(1.0, mnist_std)),
    transforms.Normalize(mean=np.multiply(-1.0, mnist_mean), std=[1.0])])

In [18]:
# Загрузка данных MNIST
mnist_temp = datasets.MNIST(root='datasets/mnist', train=True, download=True, transform=mnist_tf_train)
# Разделение тренировочного набора на тренировочный и валидационный
mnist_train, mnist_val = random_split(mnist_temp, [50000, 10000])
# Загрузка тестового набора
mnist_test = datasets.MNIST(root='datasets/mnist', train=False, download=True, transform=mnist_tf)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 464kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.14MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 14.8MB/s]


## 9

In [20]:
cifar_mean = [0.491, 0.482, 0.447] # среднее значение RGB
cifar_std = [0.202, 0.199, 0.201] # среднее отклонение
cifar_dim = 32 # размерность изображения CIFAR-10

cifar_min, cifar_max = get_clip_bounds(cifar_mean, cifar_std, cifar_dim)

In [25]:
cifar_min = cifar_min.to(device)
cifar_max = cifar_max.to(device)

In [26]:
cifar_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar_mean, std=cifar_std)])

cifar_tf_train = transforms.Compose([
    transforms.RandomCrop(size=cifar_dim, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar_mean, std=cifar_std)])

cifar_tf_inv = transforms.Compose([
    transforms.Normalize(mean=[0.0, 0.0, 0.0], std=np.divide(1.0, cifar_std)),
    transforms.Normalize(mean=np.multiply(-1.0, cifar_mean), std=[1.0, 1.0, 1.0])])

In [27]:
cifar_temp = datasets.CIFAR10(root='datasets/cifar-10', train=True, download=True, transform=cifar_tf_train)
cifar_train, cifar_val = random_split(cifar_temp, [40000, 10000])
cifar_test = datasets.CIFAR10(root='datasets/cifar-10', train=False, download=True, transform=cifar_tf)

100%|██████████| 170M/170M [00:48<00:00, 3.51MB/s]


In [28]:
cifar_classes = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck']

## 10

In [29]:
batch_size = 64
workers = 4

In [30]:
mnist_loader_train = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=workers)
mnist_loader_val = DataLoader(mnist_val, batch_size=batch_size, shuffle=False, num_workers=workers)
mnist_loader_test = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=workers)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [31]:
cifar_loader_train = DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers=workers)
cifar_loader_val = DataLoader(cifar_val, batch_size=batch_size, shuffle=False, num_workers=workers)
cifar_loader_test = DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=workers)


## 11

In [38]:
deep_args = {
    'steps':50, # Максимальное количество шагов
    'overshoot':0.02, # Параметр overshoot
    'device':device # Устройство для вычислений
}

fgsm_eps = 0.6
model = LeNet_MNIST().to(device)

model.load_state_dict(torch.load('weights/clean/mnist_lenet.pth', map_location=torch.device('cpu')))
model = model.to(device)

print("LeNet_MNIST /vs/ FGSM")
evaluate_attack('mnist_lenet_fgsm.csv', 'results', device, model, mnist_loader_test, mnist_min, mnist_max,
                fgsm_eps, is_fgsm=True)

print('\nLeNet_MNIST /vs/ DeepFool')
evaluate_attack('mnist_lenet_deepfool.csv', 'results', device, model, mnist_loader_test, mnist_min, mnist_max,
                deep_args, is_fgsm=False)

# Очистка кеша CUDA, если используется GPU
if device.type == 'cuda':
    torch.cuda.empty_cache()

LeNet_MNIST /vs/ FGSM
FGSM Test Error : 87.89%
FGSM Robustness : 4.58e-01
FGSM Time (All Images) : 0.29 s
FGSM Time (Per Image) : 28.86 us

LeNet_MNIST /vs/ DeepFool
DeepFool Test Error : 98.74%
DeepFool Robustness : 9.64e-02
DeepFool Time (All Images) : 193.32 s
DeepFool Time (Per Image) : 19.33 ms


## 12

In [39]:
fgsm_eps = 0.2
model = FC_500_150().to(device)

model.load_state_dict(torch.load('weights/clean/mnist_fc.pth', map_location=torch.device('cpu')))
model = model.to(device)

print("FC_500_150 /vs/ FGSM")
evaluate_attack('mnist_fc_fgsm.csv', 'results', device, model, mnist_loader_test, mnist_min, mnist_max,
                fgsm_eps, is_fgsm=True)

print('\nFC_500_150 /vs/ DeepFool')
evaluate_attack('mnist_fc_deepfool.csv', 'results', device, model, mnist_loader_test, mnist_min, mnist_max,
                deep_args, is_fgsm=False)

if device.type == 'cuda':
    torch.cuda.empty_cache()

FC_500_150 /vs/ FGSM
FGSM Test Error : 87.08%
FGSM Robustness : 1.56e-01
FGSM Time (All Images) : 0.15 s
FGSM Time (Per Image) : 14.99 us

FC_500_150 /vs/ DeepFool
DeepFool Test Error : 97.92%
DeepFool Robustness : 6.78e-02
DeepFool Time (All Images) : 141.81 s
DeepFool Time (Per Image) : 14.18 ms
